## Cálculo população afetada

Nesse notebook é estimada a população dos setores censitários afetada pelo buffer dos trajetos dos ônibus.

In [1]:
import pandas as pd
from utils.load_shp import load_shp
from utils.save_shp import save_shp

pd.set_option('display.max_columns', None)

In [2]:
setores_censitarios_sp = load_shp('setores_censitarios_sp.shp')

gdf_overlay = load_shp('gdf_overlay.shp')

In [3]:
setores_censitarios_sp = setores_censitarios_sp.to_crs(gdf_overlay.crs)

setores_censitarios_sp['area_setor'] = setores_censitarios_sp.geometry.area

setores_area = setores_censitarios_sp[['CD_SETOR', 'area_setor', 'pop']]

In [4]:
gdf_overlay['area_intersecao'] = gdf_overlay.geometry.area

In [5]:
gdf_overlay = gdf_overlay.merge(setores_area,
                                on = 'CD_SETOR',
                                how = 'left')

In [6]:
gdf_overlay = gdf_overlay.drop(columns = ['pop_y'])

gdf_overlay = gdf_overlay.rename(columns = {'pop_x': 'pop'})

In [7]:
gdf_overlay['pop_afetada'] = round((gdf_overlay['pop'] * (gdf_overlay['area_intersecao'] / gdf_overlay['area_setor']))).astype(int)

In [8]:
gdf_overlay = gdf_overlay.rename(columns = {'area_intersecao': 'area_inter', 
                                            'pop_afetada': 'pop_buffer'})

In [9]:
gdf_overlay.head()

,CD_SETOR,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,CD_MICRO,NM_MICRO,CD_MESO,NM_MESO,CD_RGI,NM_RGI,CD_RGINT,NM_RGINT,CD_CONCURB,NM_CONCURB,pop,v0002,v0003,v0004,v0005,v0006,v0007,cd_onibus,cd_linha,pt_inicio,pt_fim,hr_inicio,hr_fim,geometry,area_inter,area_setor,pop_buffer
0,355030801000005P,0.086820,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,35503080100,None,35061,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,579,258,258,0,2.619910,9.049774,221,31863,34938,POINT (345919.0752496967 7393358.227129335),POINT (346089.3123900441 7393132.076345487),2025-10-18 23:59:23-03:00,2025-10-19 00:00:07-03:00,"POLYGON ((339833.494 7392157.795, 339730.846 7...",10622.963101,86805.914185,71
1,355030801000007P,0.080550,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,35503080100,None,35061,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,633,299,299,0,2.416031,3.053435,262,31863,34938,POINT (345919.0752496967 7393358.227129335),POINT (346089.3123900441 7393132.076345487),2025-10-18 23:59:23-03:00,2025-10-19 00:00:07-03:00,"POLYGON ((339208.982 7392444.441, 339224.759 7...",16081.030466,80536.667630,126
2,355030801000009P,0.066178,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,35503080100,None,35061,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,487,292,292,0,2.081197,0.427350,234,31863,34938,POINT (345919.0752496967 7393358.227129335),POINT (346089.3123900441 7393132.076345487),2025-10-18 23:59:23-03:00,2025-10-19 00:00:07-03:00,"POLYGON ((339090.34 7392502.599, 339080.182 73...",448.170435,66166.932302,3
3,355030801000020P,0.063210,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,35503080100,None,35061,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,397,173,173,0,2.664430,12.080537,149,31863,34938,POINT (345919.0752496967 7393358.227129335),POINT (346089.3123900441 7393132.076345487),2025-10-18 23:59:23-03:00,2025-10-19 00:00:07-03:00,"POLYGON ((340156.189 7391993.875, 340150.09 73...",21273.418476,63199.073984,134
4,355030801000024P,0.042865,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,Água Rasa,35503080100,None,35061,São Paulo,3515,Metropolitana de São Paulo,350001,São Paulo,3501,São Paulo,3550308,São Paulo/SP,463,220,220,0,2.374359,3.076923,195,31863,34938,POINT (345919.0752496967 7393358.227129335),POINT (346089.3123900441 7393132.076345487),2025-10-18 23:59:23-03:00,2025-10-19 00:00:07-03:00,"POLYGON ((339082.164 7392466.195, 338965.985 7...",8193.662567,42858.204538,89


In [10]:
setores_uniques = pd.DataFrame(gdf_overlay['CD_SETOR'].unique())

setores_uniques.rename(columns={0:'name'}, inplace=True)

In [11]:
for i, c in setores_uniques.iterrows():
    cd = c['name']
    setor_x = gdf_overlay.loc[gdf_overlay['CD_SETOR'] == cd]
    c['soma_pop_afetada'] = sum(setor_x['pop_buffer'])
    setores_uniques.loc[setores_uniques['name'] == cd, 'subtrac_pop'] = (setor_x.loc[setor_x['CD_SETOR'] == cd, 'pop']-c['soma_pop_afetada'])

setores_uniques.sort_values(by = 'subtrac_pop', ascending = True).head(10)

,name,subtrac_pop
3,355030801000020P,263.0
7,355030801000042P,327.0
9,355030801000052P,338.0
4,355030801000024P,374.0
2,355030801000009P,484.0
1,355030801000007P,507.0
0,355030801000005P,508.0
6,355030801000040P,568.0
5,355030801000025P,649.0
8,355030801000051P,652.0


In [12]:
save_shp(gdf_overlay, 'gdf_overlay_pop_afetada.shp')

Shapefile salvo em data\gdf_overlay_pop_afetada.shp
